In [1]:
import pandas as pd

In [2]:

df = pd.read_csv("data/titanic_kaggle.csv")
df_titanic = df.sample(frac=0.75)
df_test = df.loc[df.index.difference(df_titanic),:]
    

In [3]:

from binarybeech.binarybeech import *


In [4]:

import logging
logging.basicConfig(level=logging.INFO)


In [5]:

gbt_titanic = GradientBoostedTree(df_titanic.copy(),"Survived", cart_settings={"max_depth":2,"min_leaf_samples":10, "min_split_samples":10},sample_frac=0.75,n_attributes=9
                                 )

In [6]:
gbt_titanic.X_names

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
it=gbt_titanic._initial_tree()

INFO:binarybeech.binarybeech:A tree with 2 leafs was created


In [8]:
it.tree.show()

0.3907185628742515



In [9]:
it.tree.root

In [10]:

gbt_titanic.learning_rate=0.01
gbt_titanic.create_trees(200)


INFO:binarybeech.binarybeech:A tree with 2 leafs was created
INFO:binarybeech.binarybeech:Norm of pseudo-residuals: 13.817837556236467
INFO:binarybeech.binarybeech:A tree with 8 leafs was created
INFO:binarybeech.binarybeech:Norm of pseudo-residuals: 13.761314826325277


6.27	 371.41


INFO:binarybeech.binarybeech:A tree with 8 leafs was created
INFO:binarybeech.binarybeech:Norm of pseudo-residuals: 13.715016324728403


6.21	 392.28


INFO:binarybeech.binarybeech:A tree with 8 leafs was created
INFO:binarybeech.binarybeech:Norm of pseudo-residuals: 13.666888835265517


...

-0.60	 334.72


INFO:binarybeech.binarybeech:Norm of pseudo-residuals: 10.387196450790272
INFO:binarybeech.binarybeech:A tree with 8 leafs was created


1.31	 333.78


INFO:binarybeech.binarybeech:Norm of pseudo-residuals: 10.386959498620055
INFO:binarybeech.binarybeech:A tree with 8 leafs was created


5.75	 271.07


In [11]:

gbt_titanic.predict(df_titanic.iloc[0])

0.47924083626231045

In [12]:

gbt_titanic.validate(df_test)


{'precision': array([0.8579235 , 0.77192982]),
 'recall': array([0.8579235 , 0.77192982]),
 'F-score': 0.8149266609145815,
 'accuracy': 0.8249158249158249}

#k-fold cross-validation

In [13]:

c = CART(df_titanic,"Survived",metrics_type="classification")


In [ ]:
c.train(slack=1e-3)

INFO:binarybeech.binarybeech:A tree with 113 leafs was created
INFO:binarybeech.binarybeech:A tree with 80 leafs was created
/home/jovyan/binarybeech/metrics.py:106: RuntimeWarning: invalid value encountered in true_divide
  return np.diag(m) / np.sum(m, axis=0)
INFO:binarybeech.binarybeech:A tree with 86 leafs was created


In [ ]:
c.tree.leaf_count()

In [ ]:
c.tree.show()

In [ ]:
c.validate(df_test)